# Imports

In [1]:
import numpy as np
import pandas as pd
import math

import tensorflow as tf
import keras
from tensorflow import keras
from keras import models

from utilities.InterpretationNet import *
from utilities.LambdaNet import *
from utilities.metrics import *
from utilities.utility_functions import *
from utilities.DecisionTree_BASIC import *

import utilities_LR

from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, mean_squared_error, roc_curve, auc, roc_auc_score, matthews_corrcoef
import sklearn

import matplotlib.pyplot as plt
import plotly.express as px

# Config

### Logistic Regression

In [2]:
config_LR = {
    'data': {
        'n_datasets': 45_000, # the number of datasets
        
        'n_samples': 5_000, # the number of samples per dataset
        
        'n_features': 10, 
        # The total number of features. 
        # These comprise n_informative informative features, n_redundant redundant features, n_repeated duplicated features and 
        # n_features-n_informative-n_redundant-n_repeated useless features drawn at random.
        
        #'n_informative': random.randint(2, 10),
        'n_informative': 'random',
        # The number of informative features. Each class is composed of a number of gaussian clusters each located around the vertices 
        # of a hypercube in a subspace of dimension n_informative. For each cluster, informative features are drawn independently 
        # from N(0, 1) and then randomly linearly combined within each cluster in order to add covariance. The clusters are then 
        # placed on the vertices of the hypercube.
        ### int or 'random'
        
        'n_targets': 1,
        # The number of targets (or labels) of the classification problem.
    
        'n_clusters_per_class': 1,
        # The number of clusters per class.
        
        'class_sep': 1.0,
        # class_sepfloat, default=1.0
        # The factor multiplying the hypercube size. Larger values spread out the clusters/classes and make the classification task 
        # easier.
        
        'shuffle': True,
        # Shuffle the samples and the features.
        
        'random_state': 44,
        # Determines random number generation for dataset creation. Pass an int for reproducible output across multiple function calls.
    },
    'lambda': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.1,
                'val_size': 0.15,
                'random_state': None,
                'shuffle': False, # should be always false
                'stratify': None
            }
        },
        'model_compile': {
            'optimizer_lambda': 'adam',
            'loss': 'mae',# keras.losses.BinaryCrossentropy(from_logits=False), #tf.keras.losses.get(config['lambda_net']['loss_lambda']), # 'mae'
            'metrics': ['mae', keras.losses.BinaryCrossentropy(from_logits=False)]
        },
        'model_fit': { # refer to keras API
            'batch_size': 64,
            'epochs': 500,
            'verbose': 0,
            'callbacks': None,
            'shuffle': True, # usually true
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'inets': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.1,
                'val_size': 0.15,
                'random_state': None,
                'shuffle': False,
                'stratify': None
            },
            'train_noise': 0.1 # y_flip fraction on Y_train pred data from lambda net
        },
        'model_compile': {
            
        },
        'model_fit': { # refer to keras API
            'batch_size': 256,
            'epochs': 1000,
            'verbose': 'auto',
            'callbacks': None,
            'shuffle': True,
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'computation':{
        'n_jobs': 30,
        'use_gpu': True,
        'gpu_numbers': '4',
        'RANDOM_SEED': 1,   
    }
}

# Settings

In [3]:
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'

In [4]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

os.environ['CUDA_VISIBLE_DEVICES'] = config_LR['computation']['gpu_numbers'] if config_LR['computation']['use_gpu'] else ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' if config_LR['computation']['use_gpu'] else ''

os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/local/cuda-11.4' if config_LR['computation']['use_gpu'] else ''#-10.1' #--xla_gpu_cuda_data_dir=/usr/local/cuda, 
os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=2 ,--tf_xla_enable_xla_devices' if config_LR['computation']['use_gpu'] else ''#'--tf_xla_auto_jit=2' #, --tf_xla_enable_xla_devices

In [5]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num XLA-GPUs Available: ", len(tf.config.experimental.list_physical_devices('XLA_GPU')))

Num GPUs Available:  8
Num XLA-GPUs Available:  0


# Load Models

In [6]:
@tf.function
def custom_loss(y_predictions_index, y_coef_pred):
    
    index = y_predictions_index[:, 0]
    y_true = y_predictions_index[:, 1:]
    
    # ADD NOISE TO TRAINING DATA
    noise = tf.cast(config['inets']['data_prep']['train_noise'], tf.float32)
    noise_logits = tf.tile([[1-noise, noise]], [tf.shape(y_true)[0], tf.constant(1)])
    noise_flip = tf.random.categorical(tf.math.log(noise_logits), y_true.shape[1])
    y_true = y_true*(1-noise_flip) + (1-y_true)*noise_flip   
    
    index = tf.cast(index, tf.int32)
    
    X_feature_data_samples = tf.gather(valid_feature_data, index)
    
    y_pred = tf.transpose(tf.math.add(tf.transpose(tf.linalg.matvec(X_feature_data_samples, y_coef_pred[:, 1:])), y_coef_pred[:, 0]))

    metric = tf.keras.losses.BinaryCrossentropy(
                                from_logits=True,
                                label_smoothing=0.0,
                                axis=-1,
                                reduction='auto',
                                name='binary_crossentropy')
    loss = metric(y_true, y_pred)
    return loss

In [7]:
def load_LR_inet():
    path = utilities_LR.inet_path_LR(config_LR)
    
    #model = keras.models.load_model(path + '/modelKeras', custom_objects={'custom_loss': custom_loss})
    model = keras.models.load_model(path + '/modelKeras' + '.h5' , custom_objects={'custom_loss': custom_loss})
    print(path)
    return model

In [8]:
model_LR = load_LR_inet()

data_LR/nda45000_nsa5000_nfe10_ninrandom_nta1_ncc1_sep1.0_shuTrue_ran44/tsi0.1_vsi0.15_ranNone_shuFalse_strNone_bat64_epo500_shuTrue_claNone_samNone_ini0_steNone_vstNone_vbsNone_vfr1/tsi0.1_vsi0.15_noi0.1_ranNone_shuFalse_strNone_bat256_epo1000_shuTrue_claNone_samNone_ini0_steNone_vstNone_vbsNone_vfr1


In [9]:
model_LR.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2048)              1314816   
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense_1 (Dense)             (None, 1024)              2098176   
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 dense_2 (Dense)             (None, 512)               524800    
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 11)                5

# Load Testing Data

In [10]:
def get_y_pred_data():
    directory = utilities_LR.lambda_path_LR(config_LR)
    
    return np.load(directory + '/lambda_preds_list.npy', allow_pickle=True)

In [11]:
y_predictions_from_lambda = get_y_pred_data()

In [12]:
def load_lambda():
    directory = utilities_LR.lambda_path_LR(config_LR)
    
    return np.load(directory + '/lambda_weights_list.npy', allow_pickle=True)

In [13]:
lambda_weights = load_lambda()

In [14]:
directory = utilities_LR.data_path_LR(config_LR)

with open(directory + '/X.npy', "rb") as f:
    X_datasets_list_LR_test = np.load(f, allow_pickle=True)

In [15]:
_, X_datasets_list_LR_test, _, y_predictions_from_lambda, _, lambda_weights = train_test_split(X_datasets_list_LR_test, 
                                                    y_predictions_from_lambda, 
                                                    lambda_weights,
                                                    test_size=config_LR['inets']['data_prep']['train_test_val_split']['test_size'], 
                                                    random_state=config_LR['inets']['data_prep']['train_test_val_split']['random_state'], 
                                                    shuffle=config_LR['inets']['data_prep']['train_test_val_split']['shuffle'], 
                                                    stratify=config_LR['inets']['data_prep']['train_test_val_split']['stratify'])

# Evaluate Inet for LR

In [16]:
def precision(tp, fp, tn, fn):
    return tp / (tp + fp)

In [17]:
def recall(tp, fp, tn, fn):
    return tp / (tp + fn)

In [18]:
def f1(tp, fp, tn, fn):
    pre = precision(tp, fp, tn, fn)
    rec = recall(tp, fp, tn, fn)
    return 2 * ((pre * rec) / (pre + rec))

# Get Predictions Inet

In [19]:
classifications_true_pred = np.zeros([X_datasets_list_LR_test.shape[0], config_LR['data']['n_samples'], 2]) ## binary classification
results_np = np.zeros([X_datasets_list_LR_test.shape[0], 11])

In [20]:
for index, (X, y_pred_from_lambda, lambda_weight) in enumerate(zip(X_datasets_list_LR_test, y_predictions_from_lambda, lambda_weights), start=0):
    lambda_weight = lambda_weight.reshape([1, lambda_weight.shape[0]])
    coef_pred_inet = model_LR.predict(lambda_weight, verbose=0)
    coef_pred_inet = coef_pred_inet.reshape([config_LR['data']['n_features'] + 1])
    
    y_pred = np.add(coef_pred_inet[0], np.dot(X, coef_pred_inet[1:]))
    y_pred = keras.activations.sigmoid(y_pred).numpy()
    y_pred = y_pred.reshape([config_LR['data']['n_samples'], 1])
    
    y_true = y_pred_from_lambda
    y_true = y_true.reshape([config_LR['data']['n_samples'], 1])
    y_true = y_true.round().astype(int)
    
    classifications_true_pred[index] = np.concatenate([y_true, y_pred], axis=1)
    
    if index % 100 == 0:
        print("done", index)

done 0
done 100
done 200
done 300
done 400


KeyboardInterrupt: 

In [21]:
y_true_all = classifications_true_pred[:, :, 0].flatten()
y_pred_all = classifications_true_pred[:, :, 1].flatten()

## ROC Curve

In [22]:
fpr, tpr, threshold = roc_curve(y_true_all, y_pred_all, drop_intermediate=False)

In [23]:
save_fprtpr = np.array([fpr, tpr, threshold])

In [24]:
path = utilities_LR.inet_path_LR(config_LR)

np.save(path + f"/fprTprThreshold_valid_n{config_LR['data']['n_features']}_noise{config_LR['inets']['data_prep']['train_noise']}", save_fprtpr, allow_pickle=True)

## F1

In [25]:
threshold_valid = threshold[np.argmax(tpr-fpr)]

In [26]:
threshold_valid

0.49483271388667616

In [27]:
for i in range(classifications_true_pred.shape[0]):
    x = classifications_true_pred[i]
    y_true = x[:, 0]
    y_pred = x[:, 1]
    y_pred = np.where(y_pred > threshold_valid, 1, 0).astype(int)
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    mse = mean_squared_error(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    
    roc_score=-1
    
    results_np[i] = [i, mse, tn, fp, fn, tp, precision(tp, fp, tn, fn), recall(tp, fp, tn, fn), f1(tp, fp, tn, fn), roc_score, mcc]

In [28]:
result = pd.DataFrame(columns=["index_0=aggregated", "mse",  "tn", "fp", "fn", "tp", "precision", "recall", "f1", "ROC-AUC", "MCC"], data=results_np)

In [29]:
aggragated = pd.DataFrame(result.mean(numeric_only=False)).transpose()

In [30]:
aggragated

,index_0=aggregated,mse,tn,fp,fn,tp,precision,recall,f1,ROC-AUC,MCC
0,2249.5,0.176975,2052.52,447.406667,437.468,2062.605333,0.837331,0.824903,0.818142,-1.0,0.660174


In [31]:
aggragated.at[0, "index_0=aggregated"] = 0
result_aggregated = pd.concat([aggragated, result], axis=0, ignore_index=True)

In [32]:
def save_eval_res_valid(df):
    path = utilities_LR.inet_path_LR(config_LR)
    
    model = df.to_csv(path + f"/evalRes_valid_n{config_LR['data']['n_features']}_noise{config_LR['inets']['data_prep']['train_noise']}.csv")
    print(path)

In [33]:
save_eval_res_valid(result_aggregated)

data_LR/nda45000_nsa5000_nfe10_ninrandom_nta1_ncc1_sep1.0_shuTrue_ran44/tsi0.1_vsi0.15_ranNone_shuFalse_strNone_bat64_epo500_shuTrue_claNone_samNone_ini0_steNone_vstNone_vbsNone_vfr1/tsi0.1_vsi0.15_noi0.1_ranNone_shuFalse_strNone_bat256_epo1000_shuTrue_claNone_samNone_ini0_steNone_vstNone_vbsNone_vfr1


# For Comparison: Plain LogReg

In [21]:
split = 0.15

In [35]:
classifications_true_pred_plain_logreg = np.zeros([X_datasets_list_LR_test.shape[0], int(config_LR['data']['n_samples']*split), 2]) ## binary classification
results_np_plain_logreg = np.zeros([X_datasets_list_LR_test.shape[0], 11])

In [36]:
for i, (X, y_pred_from_lambda) in enumerate(zip(X_datasets_list_LR_test, y_predictions_from_lambda), start=0):
    
    y_true = y_pred_from_lambda
    y_true = y_true.round().astype(int)
    
    X, X_test, y_true, y_true_test = train_test_split(X, y_true, test_size=split)
    
    noise = config_LR['inets']['data_prep']['train_noise']
    index = np.random.choice(y_true_test.shape[0], size=int((y_true_test.shape[0] * noise)), replace=False)
    y_true_test[index] = [1 - x for x in y_true_test[index]]
    

    logreg = LogisticRegression()
    logreg.fit(X, y_true)
    
    y_pred = logreg.predict_proba(X_test)
    y_pred = y_pred[:, 1]
    
    y_pred = y_pred.reshape([y_pred.shape[0], 1])
    y_true_test = y_true_test.reshape([y_true_test.shape[0], 1])
    
    classifications_true_pred_plain_logreg[i] = np.concatenate([y_true_test, y_pred], axis=1)
    
    if i % 100 == 0:
        print("done", i)

done 0
done 100
done 200
done 300
done 400
done 500
done 600
done 700
done 800
done 900
done 1000
done 1100
done 1200
done 1300
done 1400
done 1500
done 1600
done 1700
done 1800
done 1900
done 2000
done 2100
done 2200
done 2300
done 2400
done 2500
done 2600
done 2700
done 2800
done 2900
done 3000
done 3100
done 3200
done 3300
done 3400
done 3500
done 3600
done 3700
done 3800
done 3900
done 4000
done 4100
done 4200
done 4300
done 4400


In [37]:
y_true_all_plain_logreg = classifications_true_pred_plain_logreg[:, :, 0].flatten()
y_pred_all_plain_logreg = classifications_true_pred_plain_logreg[:, :, 1].flatten()

## ROC Curve

In [38]:
fpr_plain_logreg, tpr_plain_logreg, threshold_plain_logreg = roc_curve(y_true_all_plain_logreg, y_pred_all_plain_logreg, drop_intermediate=False)

In [39]:
save_fprtpr_plain_logreg = np.array([fpr_plain_logreg, tpr_plain_logreg, threshold_plain_logreg])

In [40]:
path = utilities_LR.inet_path_LR(config_LR)

np.save(path + f"/fprTprThreshold_plain_logreg_n{config_LR['data']['n_features']}_noise{config_LR['inets']['data_prep']['train_noise']}", save_fprtpr_plain_logreg, allow_pickle=True)

## F1

In [41]:
threshold_plain_logreg = threshold_plain_logreg[np.argmax(tpr_plain_logreg-fpr_plain_logreg)]

In [42]:
threshold_plain_logreg

0.4875662677052132

In [43]:
for i in range(classifications_true_pred_plain_logreg.shape[0]):
    x = classifications_true_pred_plain_logreg[i]
    y_true = x[:, 0]
    y_pred = x[:, 1]
    y_pred = np.where(y_pred > threshold_plain_logreg, 1, 0).astype(int)
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    mse = mean_squared_error(y_true, y_pred)
    roc_score = roc_auc_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    
    results_np_plain_logreg[i] =  [i, mse, tn, fp, fn, tp, precision(tp, fp, tn, fn), recall(tp, fp, tn, fn), f1(tp, fp, tn, fn), roc_score, mcc]

In [44]:
result_plain_logreg = pd.DataFrame(columns=["index_0=aggregated", "mse",  "tn", "fp", "fn", "tp", "precision", "recall", "f1", "ROC-AUC", "MCC"], data=results_np_plain_logreg)

In [45]:
aggragated_plain_logreg = pd.DataFrame(result_plain_logreg.mean(numeric_only=False)).transpose()

In [46]:
aggragated_plain_logreg

,index_0=aggregated,mse,tn,fp,fn,tp,precision,recall,f1,ROC-AUC,MCC
0,2249.5,0.142892,320.665111,54.351556,52.817778,322.165556,0.856106,0.859113,0.857288,0.857062,0.714449


In [47]:
aggragated_plain_logreg.at[0, "index_0=aggregated"] = 0
result_aggregated_plain_logreg = pd.concat([aggragated_plain_logreg, result_plain_logreg], axis=0, ignore_index=True)

In [48]:
def save_eval_res_plain_logreg(df):
    path = utilities_LR.inet_path_LR(config_LR)
    
    model = df.to_csv(path + f"/evalRes_plain_logreg_n{config_LR['data']['n_features']}_noise{config_LR['inets']['data_prep']['train_noise']}.csv")
    print(path)

In [49]:
save_eval_res_plain_logreg(result_aggregated_plain_logreg)

data_LR/nda45000_nsa5000_nfe10_ninrandom_nta1_ncc1_sep1.0_shuTrue_ran44/tsi0.1_vsi0.15_ranNone_shuFalse_strNone_bat64_epo500_shuTrue_claNone_samNone_ini0_steNone_vstNone_vbsNone_vfr1/tsi0.1_vsi0.15_noi0.1_ranNone_shuFalse_strNone_bat256_epo1000_shuTrue_claNone_samNone_ini0_steNone_vstNone_vbsNone_vfr1


# For Comparison: Plain Decision Trees

In [22]:
classifications_true_pred_plain_DT = np.zeros([X_datasets_list_LR_test.shape[0], int(config_LR['data']['n_samples']*split), 2]) ## binary classification
results_np_plain_DT = np.zeros([X_datasets_list_LR_test.shape[0], 11])

In [23]:
for i, (X, y_pred_from_lambda) in enumerate(zip(X_datasets_list_LR_test, y_predictions_from_lambda), start=0):
    
    y_true = y_pred_from_lambda
    y_true = y_true.round().astype(int)
    
    X, X_test, y_true, y_true_test = train_test_split(X, y_true, test_size=split)
    
    noise = config_LR['inets']['data_prep']['train_noise']
    index = np.random.choice(y_true_test.shape[0], size=int((y_true_test.shape[0] * noise)), replace=False)
    y_true_test[index] = [1 - x for x in y_true_test[index]]
    
    dt = tree.DecisionTreeClassifier(max_depth=4)
    dt.fit(X, y_true)
    
    y_pred = dt.predict_proba(X_test)
    y_pred = y_pred[:, 1]
    
    y_pred = y_pred.reshape([y_pred.shape[0], 1])
    y_true_test = y_true_test.reshape([y_true_test.shape[0], 1])
    
    classifications_true_pred_plain_DT[i] = np.concatenate([y_true_test, y_pred], axis=1)
    
    if i % 100 == 0:
        print("done", i)

done 0
done 100
done 200
done 300
done 400
done 500
done 600
done 700
done 800
done 900
done 1000
done 1100
done 1200
done 1300
done 1400
done 1500
done 1600
done 1700
done 1800
done 1900
done 2000
done 2100
done 2200
done 2300
done 2400
done 2500
done 2600
done 2700
done 2800
done 2900
done 3000
done 3100
done 3200
done 3300
done 3400
done 3500
done 3600
done 3700
done 3800
done 3900
done 4000
done 4100
done 4200
done 4300
done 4400


In [24]:
y_true_all_plain_DT = classifications_true_pred_plain_DT[:, :, 0].flatten()
y_pred_all_plain_DT = classifications_true_pred_plain_DT[:, :, 1].flatten()

## ROC Curve

In [25]:
fpr_plain_DT, tpr_plain_DT, threshold_plain_DT = roc_curve(y_true_all_plain_DT, y_pred_all_plain_DT, drop_intermediate=False)

In [26]:
save_fprtpr_plain_DT = np.array([fpr_plain_DT, tpr_plain_DT, threshold_plain_DT])

In [27]:
path = utilities_LR.inet_path_LR(config_LR)

np.save(path + f"/fprTprThreshold_plain_DT_n{config_LR['data']['n_features']}_noise{config_LR['inets']['data_prep']['train_noise']}", save_fprtpr_plain_DT, allow_pickle=True)

## F1

In [28]:
threshold_plain_DT = threshold_plain_DT[np.argmax(tpr_plain_DT-fpr_plain_DT)]

In [29]:
threshold_plain_DT

0.5130434782608696

In [30]:
for i in range(classifications_true_pred_plain_DT.shape[0]):
    x = classifications_true_pred_plain_DT[i]
    y_true = x[:, 0]
    y_pred = x[:, 1]
    y_pred = np.where(y_pred > threshold_plain_DT, 1, 0).astype(int)
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    mse = mean_squared_error(y_true, y_pred)
    roc_score = roc_auc_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    
    results_np_plain_DT[i] = [i, mse, tn, fp, fn, tp, precision(tp, fp, tn, fn), recall(tp, fp, tn, fn), f1(tp, fp, tn, fn), roc_score, mcc]

In [31]:
result_plain_DT = pd.DataFrame(columns=["index_0=aggregated", "mse",  "tn", "fp", "fn", "tp", "precision", "recall", "f1", "ROC-AUC", "MCC"], data=results_np_plain_DT)

In [32]:
aggragated_plain_DT = pd.DataFrame(result_plain_DT.mean(numeric_only=False)).transpose()

In [33]:
aggragated_plain_DT

,index_0=aggregated,mse,tn,fp,fn,tp,precision,recall,f1,ROC-AUC,MCC
0,2249.5,0.165944,313.765778,61.640889,62.816889,311.776444,0.835795,0.832187,0.833167,0.833964,0.668999


In [34]:
aggragated_plain_DT.at[0, "index_0=aggregated"] = 0
result_aggregated_plain_DT = pd.concat([aggragated_plain_DT, result_plain_DT], axis=0, ignore_index=True)

In [35]:
def save_eval_res_plain_DT(df):
    path = utilities_LR.inet_path_LR(config_LR)
    
    model = df.to_csv(path + f"/evalRes_plain_DT_n{config_LR['data']['n_features']}_noise{config_LR['inets']['data_prep']['train_noise']}.csv")
    print(path)

In [36]:
save_eval_res_plain_DT(result_aggregated_plain_DT)

data_LR/nda45000_nsa5000_nfe10_ninrandom_nta1_ncc1_sep1.0_shuTrue_ran44/tsi0.1_vsi0.15_ranNone_shuFalse_strNone_bat64_epo500_shuTrue_claNone_samNone_ini0_steNone_vstNone_vbsNone_vfr1/tsi0.1_vsi0.15_noi0.1_ranNone_shuFalse_strNone_bat256_epo1000_shuTrue_claNone_samNone_ini0_steNone_vstNone_vbsNone_vfr1
